<a href="https://colab.research.google.com/github/BustamJos3/ia4eng_jdbs_afmp_project/blob/main/01_exploracion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from zipfile import ZipFile

In [ ]:
#call API
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bustamjos3","key":"a80d020fd1917115be9ce5ce749b71d9"}'}

In [ ]:
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#API from forest cover dataset
!kaggle competitions download -c forest-cover-type-prediction

 95% 24.0M/25.3M [00:00<00:00, 120MB/s]
100% 25.3M/25.3M [00:00<00:00, 121MB/s]


In [ ]:
#unzip files
file_name = 'forest-cover-type-prediction.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
#create DF from train
df_train = pd.read_csv('train.csv')

In [ ]:
#description of df_train from 1st column to 5 column
df_train.describe().loc[:,:df_train.describe().columns[4]]

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology
count,15120.00000,15120.000000,15120.000000,15120.000000,15120.000000
mean,7560.50000,2749.322553,156.676653,16.501587,227.195701
std,4364.91237,417.678187,110.085801,8.453927,210.075296
min,1.00000,1863.000000,0.000000,0.000000,0.000000
25%,3780.75000,2376.000000,65.000000,10.000000,67.000000
50%,7560.50000,2752.000000,126.000000,15.000000,180.000000
75%,11340.25000,3104.000000,261.000000,22.000000,330.000000
max,15120.00000,3849.000000,360.000000,52.000000,1343.000000
